In [1]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

In [2]:
V = TypeVar('V')
D = TypeVar('D') 

In [3]:
class Constraint(Generic[V, D], ABC):
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    @abstractmethod
    def is_satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

In [4]:
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]):
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it")

    def add_constraint(self, constraint: Constraint[V, D]):
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    def consistent(self, variable: V, assignment: Dict[V, D]):
        for constraint in self.constraints[variable]:
            if not constraint.is_satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        if len(assignment) == len(self.variables):
            return assignment

        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        first: V = self.mcv_algorithm(unassigned)
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                if result is not None:
                    return result
        return None

    def mcv_algorithm(self, unassigned_vars):
        mcvarible= unassigned_vars[0]
        for variable in unassigned_vars:
            if len(self.constraints[variable]) > len(self.constraints[mcvarible]):
                mcvarible = variable
        return mcvarible

In [5]:
def get_animal(animal):
    switcher={          
        "Leao": "Leao",
        "Antilope": "Antilope",
        "Tigre": "Tigre",
        "Hiena": "Hiena",
        "Pavao": "Pavao",
        "Suricate": "Suricate",
        "Javali": "Javali"
    }

In [6]:
def get_animal2(animal):
    if animal == "Leao":
        print("Leão")
    elif animal == "Tigre":
        print("Tigre")
    elif animal == "Antilope":
        print("Antilope")
    elif animal == "Hiena":
        print("Hiena")
    elif animal == "Pavao":
        print("Pavão")
    elif animal == "Suricate":
        print("Suricate")
    elif animal == "Javali":
        print("Javali")

In [7]:
class Zoo_constraint(Constraint[str, str]):
    def __init__(self, animal1: str, animal2: str):
        super().__init__([animal1, animal2])
        self.animal1 = animal1
        self.animal2 = animal2

    def is_satisfied(self, assignment: Dict[str, int]):
        if self.animal1 == "Leao" and self.animal1 in assignment:
            return assignment[self.animal1] == 1
        if self.animal1 == "Antilope" and self.animal1 in assignment:
            if "Leao" in assignment and "Tigre" in assignment:
                return assignment[self.animal1] != (assignment["Leao"] + 1) and assignment[self.animal1] != (assignment["Tigre"] + 1) and assignment[self.animal1] != (assignment["Tigre"] - 1) 
        if self.animal1 not in assignment or self.animal2 not in assignment:
            return True
        return assignment[self.animal1] != assignment[self.animal2]

In [10]:
class Animal_distribution():
    variables: List[str] = ["Tigre", "Antilope", "Hiena","Javali" , "Leao", "Pavao", "Suricate"]
    domains: Dict[str, List[int]] = {}
    for variable in variables:
        domains[variable] = [1, 2, 3, 4]
    csp: CSP[str, str] = CSP(variables, domains)
    csp.add_constraint(Zoo_constraint("Leao", "Tigre"))
    csp.add_constraint(Zoo_constraint("Leao", "Pavao"))
    csp.add_constraint(Zoo_constraint("Leao", "Antilope"))
    csp.add_constraint(Zoo_constraint("Antilope", "Leao"))
    csp.add_constraint(Zoo_constraint("Antilope", "Tigre"))
    csp.add_constraint(Zoo_constraint("Hiena", "Leao"))
    csp.add_constraint(Zoo_constraint("Hiena", "Antilope"))
    csp.add_constraint(Zoo_constraint("Hiena", "Javali"))
    csp.add_constraint(Zoo_constraint("Hiena", "Pavao"))
    csp.add_constraint(Zoo_constraint("Hiena", "Suricate"))
    csp.add_constraint(Zoo_constraint("Tigre", "Pavao"))
    csp.add_constraint(Zoo_constraint("Tigre", "Suricate"))
    csp.add_constraint(Zoo_constraint("Tigre", "Javali"))
    csp.add_constraint(Zoo_constraint("Tigre", "Antilope"))
    csp.add_constraint(Zoo_constraint("Tigre", "Leao"))
    csp.add_constraint(Zoo_constraint("Pavao", "Leao"))
    csp.add_constraint(Zoo_constraint("Pavao", "Tigre"))
    csp.add_constraint(Zoo_constraint("Suricate", "Tigre"))
    csp.add_constraint(Zoo_constraint("Suricate", "Hiena"))
    csp.add_constraint(Zoo_constraint("Javali", "Tigre"))
    csp.add_constraint(Zoo_constraint("Javali", "Hiena"))
    solution: Optional[Dict[str, str]] = csp.backtracking_search()
    if solution is None:
        print("No solution found!")
    else:
        print("--------")
        for cage in [1, 2, 3, 4]:
            print("Cage", cage)
            for animal, zone in solution.items():
                if zone == cage:
                    get_animal2(animal)
            print("--------")

--------
Cage 1
Leão
Javali
Suricate
--------
Cage 2
Tigre
Hiena
--------
Cage 3
Pavão
--------
Cage 4
Antilope
--------
